In [1]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import NoSuchElementException,TimeoutException,ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [2]:
options = Options()
options.binary_location = '/usr/bin/google-chrome'
chromedriver_path='../Driver/chromedriver_c'
options.add_argument(f"webdriver.chrome.driver={chromedriver_path}")

 
# Adding argument to disable the AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 
#options.add_experimental_option("prefs", { "profile.default_content_setting_values.geolocation": 1})
options.add_argument("--disable-notifications")  # Disable notifications
 
# Exclude the collection of enable-automation switches 
#options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# Turn-off userAutomationExtension 
#options.add_experimental_option("useAutomationExtension", False) 
 
# Setting the driver path and requesting a page 
#driver = uc.Chrome(options=options)
driver = uc.Chrome(driver_executable_path=ChromeDriverManager().install(),options=options)

# Changing the property of the navigator value for webdriver to undefined 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 

In [3]:
website="https://naivas.online/"
website="https://naivas.online/beauty-cosmetics"
#website="..Driver/testpages/Naivas Kenya _ Everything You Love On Demand.html"
#website="https://naivas.online/hardware"
#website="https://naivas.online/standard-bike"
#website="https://naivas.online/baby-kids"
#main_cat="PET"
#website="https://naivas.online/stationery"


In [4]:
def rejectCookies():
    shadowxpath='//div[@id="axeptio_overlay"]/div'
    disable_cookie=WebDriverWait(driver,40).until(EC.presence_of_element_located((By.XPATH,shadowxpath))).shadow_root
    no=WebDriverWait(disable_cookie,40).until(EC.visibility_of_element_located((By.ID,"axeptio_btn_dismiss")))
    actions=ActionChains(driver)
    actions.move_to_element(no).perform()
    no.click()

In [5]:
def menuLoop():
    shop_menu_xpath='//li/a[@href="https://naivas.online/"]'
    wait=WebDriverWait(driver,30)
    shop_menu=wait.until(EC.presence_of_element_located((By.XPATH,shop_menu_xpath)))
    action=ActionChains(driver)
    action.move_to_element(shop_menu).perform()
    menu=driver.find_element(By.XPATH,'//ul[contains(@class,"mm_menus_ul")]/li[2]')                                                       
    categories1=menu.find_elements(By.XPATH,'//li[contains(@class,"mm_tabs_li mm_tabs_has_content")]')
    time.sleep(5)
    print(len(categories1))
    for cat in categories1:
        action.move_to_element(cat).perform()
        time.sleep(5)

In [6]:
def scrollToFooter():
    action=ActionChains(driver)
    action.move_to_element(driver.find_element(By.XPATH,'//div[@id="blockEmailSubscription_displayFooterRight"]/p')).perform()

In [7]:
def isDiscounted(product):
    try:
        product.find_element(By.XPATH,'.//span[@class="product-flag discount"]')
        return True
    except NoSuchElementException:
        return False

In [8]:
def getProductList():
    ps=driver.find_elements(By.XPATH,'//div[@id="js-product-list"]//div[@class="product-miniature js-product-miniature"]')
    return ps

In [9]:
def getProductDetails(prod,category="",sub_cat="",brand=""):
    product={"Name":"","Price":"","Discount Price":"","Category":category,"Sub Category":sub_cat,"Brand":brand}
    #product["Main Category"]=main
    #product["Sub Category"]=sub
    product["Name"]=prod.find_element(By.XPATH,'.//h5[@class="product-name"]/a').get_attribute("title")
    product["id"]=prod.get_attribute("data-id-product")
    
    try:
        if isDiscounted(prod):
            product["Price"]=prod.find_element(By.XPATH,'.//span[@class="regular-price"]').text
            product["Discount Price"]=prod.find_element(By.XPATH,'.//span[@class="price product-price"]').text
        else:
            product["Price"]=prod.find_element(By.XPATH,'.//span[@class="price product-price"]').text
            product["Discount Price"]=product["Price"]
        return product
    except NoSuchElementException as nse:
        print(product["Name"])
        print(str(nse))
        product["Price"]=None
        product["Discount Price"]=None
        return product
        
        
    
    

In [10]:
def productCount():
    try:
        #count_string=driver.find_element(By.XPATH,'//div[@id="js-product-list-top"]//div/p[contains(text(),"There")]').text
        count_string=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//div[@id="js-product-list-top"]//div/p[contains(text(),"There")]'))).text
        count_string=count_string.replace("There ","")
        count_string=count_string.replace("is ","")
        count_string=count_string.replace("are ","")
        count_string=count_string.replace(" product.","")
        count_string=count_string.replace(" products.","")
        count=int(count_string)
        return count
    except TimeoutException:
        return 0

In [11]:
def hasCategories():
    try:
        cat_xpath='//div[@id="_desktop_search_filters"]//h5[contains(text(),"Categories")]'
        WebDriverWait(driver,15).until(EC.visibility_of_element_located((By.XPATH,cat_xpath)))
        return True
    except TimeoutException:
        return False

In [12]:
def hasBrands():
    try:
        brand_xpath='//div[@id="_desktop_search_filters"]//h5[contains(text(),"Brand")]'
        WebDriverWait(driver,15).until(EC.visibility_of_element_located((By.XPATH,brand_xpath)))
        return True
    except TimeoutException:
        return False

In [13]:
def getCategories():
    cat_xpath='//div[@id="_desktop_search_filters"]//h5[contains(text(),"Categories")]/..//li//a'
    sub_cats=driver.find_elements(By.XPATH,cat_xpath)
    return sub_cats

In [14]:
def getBrands():
    brand_xpath='//div[@id="_desktop_search_filters"]//h5[contains(text(),"Brand")]/..//li//a'
    brands=driver.find_elements(By.XPATH,brand_xpath)
    return brands

In [15]:
def getProductsWithBrand():
    if hasBrands():
                brands=getBrands()
                for b_index in range(len(brands)):
                    brand=brands[b_index]
                    ActionChains(driver).move_to_element(brand).perform()
                    brand_name=brand.text
                    if sub_cat+brand_name in complete:
                        continue
                    print(f"\t{brand_name}")
                    brand.click()
                    time.sleep(15)
                    scraped_data.extend(scrapeData(main_cat,sub_cat,brand_name))
                    complete.append(sub_cat+brand_name)
                    brands=getBrands()
                    ActionChains(driver).move_to_element(brands[b_index]).perform()
                    brands[b_index].click()
                    time.sleep(15)
                    brands=getBrands()
    else:
                scraped_data.extend(scrapeData(main_cat,sub_cat))

In [16]:
def filterThrough(scraped_data,complete,focus,check):
    if hasCategories():
        categories=getCategories()
        for index in range(focus,len(categories)):  
            cat=categories[index]
            ActionChains(driver).move_to_element(cat).perform()
            sub_cat=cat.text
            print(sub_cat)
            if check:
                cat.click()
                #load with cat specific brands
                time.sleep(15)
            if hasBrands():
                brands=getBrands()
                for b_index in range(len(brands)):
                    brand=brands[b_index]
                    ActionChains(driver).move_to_element(brand).perform()
                    brand_name=brand.text
                    if sub_cat+brand_name in complete:
                        continue
                    print(f"\t{brand_name}")
                    brand.click()
                    time.sleep(15)
                    scraped_data.extend(scrapeData(main_cat,sub_cat,brand_name))
                    complete.append(sub_cat+brand_name)
                    brands=getBrands()
                    ActionChains(driver).move_to_element(brands[b_index]).perform()
                    brands[b_index].click()
                    time.sleep(15)
                    brands=getBrands()
            else:
                scraped_data.extend(scrapeData(main_cat,sub_cat))
            categories=getCategories()
            complete.append(categories[index].text)
            ActionChains(driver).move_to_element(categories[index]).perform()
            categories[index].click()
            time.sleep(15)
            categories=getCategories()
            check=True
            focus+=1
            print(check,focus)
    else:
        scraped_data.extend(scrapeData(main_cat))

In [17]:
def loadAll():
    number_of_products=productCount()
    plist=getProductList()
    while len(plist)!=number_of_products:
        scrollToFooter()
        time.sleep(5)
        plist=getProductList()

In [18]:
def scrapeData(category="",sub_cat="",brand=""):
    products=[]
    if productCount()==0:
        return []
    else:
        loadAll()
        for prod in getProductList():
            product=getProductDetails(prod,category,sub_cat,brand)
            products.append(product)
        return products

In [19]:
driver.get(website)
rejectCookies()
time.sleep(20)

In [20]:
scraped_data,complete,focus,check=[],[],0,True

In [21]:

try:
    filterThrough(scraped_data,complete,focus,check)
except ElementClickInterceptedException:
    check=False
    driver.refresh()
    time.sleep(20)
  

BODY CARE
	ALISON
	AMARA
	ARIMIS
	BALLET
	BIO
	IMPERIAL LEATHER
	INGRAM
	LURON
	NICE AND LOVELY
	NIVEA
	TROPICAL
	VALON
	VASELINE
	VERSMAN
	ZOE
True 1
COTTON WOOL & BUDS
	DOVE
	TENA
	TENDERCARE
	TIPPYS
	VELVEX
True 2
DEODORANTS & ANTIPERSPIRANTS
	AXE
	CHRIS ADAMS
	FA
	LURON
	NIVEA
	RASASI
	REXONA
True 3
FACE CARE
	ACNES
	ARYUV
	BELLA
	DETTOL
	GARNIER
	HANAN
	LA FEMME
	LURON
	NIVEA
	SIMPLE
	ST.IVES


In [22]:
focus,check,complete


(0,
 False,
 ['BODY CAREALISON',
  'BODY CAREAMARA',
  'BODY CAREARIMIS',
  'BODY CAREBALLET',
  'BODY CAREBIO',
  'BODY CAREIMPERIAL LEATHER',
  'BODY CAREINGRAM',
  'BODY CARELURON',
  'BODY CARENICE AND LOVELY',
  'BODY CARENIVEA',
  'BODY CARETROPICAL',
  'BODY CAREVALON',
  'BODY CAREVASELINE',
  'BODY CAREVERSMAN',
  'BODY CAREZOE',
  'BODY CARE',
  'COTTON WOOL & BUDSDOVE',
  'COTTON WOOL & BUDSTENA',
  'COTTON WOOL & BUDSTENDERCARE',
  'COTTON WOOL & BUDSTIPPYS',
  'COTTON WOOL & BUDSVELVEX',
  'COTTON WOOL & BUDS',
  'DEODORANTS & ANTIPERSPIRANTSAXE',
  'DEODORANTS & ANTIPERSPIRANTSCHRIS ADAMS',
  'DEODORANTS & ANTIPERSPIRANTSFA',
  'DEODORANTS & ANTIPERSPIRANTSLURON',
  'DEODORANTS & ANTIPERSPIRANTSNIVEA',
  'DEODORANTS & ANTIPERSPIRANTSRASASI',
  'DEODORANTS & ANTIPERSPIRANTSREXONA',
  'DEODORANTS & ANTIPERSPIRANTS',
  'FACE CAREACNES',
  'FACE CAREARYUV',
  'FACE CAREBELLA',
  'FACE CAREDETTOL',
  'FACE CAREGARNIER',
  'FACE CAREHANAN',
  'FACE CARELA FEMME',
  'FACE CARELU

In [23]:
check=False
focus,check

(0, False)

In [31]:
df=pd.DataFrame(scraped_data)
df

,Name,Price,Discount Price,Category,Sub Category,Brand,id
0,Alison's Ideal Pure Castor Oil 50ml,KES 141.00,KES 141.00,PET,BODY CARE,ALISON,12337
1,Alison'S Ideal Pure Castor Oil 200ml,KES 392.00,KES 392.00,PET,BODY CARE,ALISON,12336
2,Alison's Ideal Pure Castor Oil 100ml,KES 230.00,KES 230.00,PET,BODY CARE,ALISON,12335
3,Alison's Ideal Olive Oil 25ml,KES 101.00,KES 101.00,PET,BODY CARE,ALISON,12333
4,Alison's Coconut & Olive Oil 200g,KES 294.00,KES 294.00,PET,BODY CARE,ALISON,12331
...,...,...,...,...,...,...,...
312,Garnier Pure Active 3 In 1 150ml,"KES 1,295.00","KES 1,295.00",PET,FACE CARE,ST.IVES,12555
313,Garnier Hydra Bomb Pomeg Mask 32g,KES 350.00,KES 350.00,PET,FACE CARE,ST.IVES,12554
314,Garnier Hydra Bomb Green T Mask 32g,KES 350.00,KES 350.00,PET,FACE CARE,ST.IVES,12553
315,Garnier Hydra Bomb Chamomi Mask 32g,KES 350.00,KES 350.00,PET,FACE CARE,ST.IVES,12552


In [25]:
#df.to_csv(f"naivas_{main_cat}.csv",index=False)

In [26]:
#df.duplicated(subset=["id"]).index

In [27]:
#df[df.duplicated(subset=["id"],keep=False)].index

In [28]:
#df[df.duplicated(subset=["id"],keep=False)]

In [29]:
#df=df.drop(df[df.duplicated(subset=["id"],keep="first")].index)

In [30]:
#df.duplicated(subset=["id"]).sum()

In [ ]:
complete=['BODY CAREALISON',
  'BODY CAREAMARA',
  'BODY CAREARIMIS',
  'BODY CAREBALLET',
  'BODY CAREBIO',
  'BODY CAREIMPERIAL LEATHER',
  'BODY CAREINGRAM',
  'BODY CARELURON',
  'BODY CARENICE AND LOVELY',
  'BODY CARENIVEA',
  'BODY CARETROPICAL',
  'BODY CAREVALON',
  'BODY CAREVASELINE',
  'BODY CAREVERSMAN',
  'BODY CAREZOE',
  'BODY CARE',
  'COTTON WOOL & BUDSDOVE',
  'COTTON WOOL & BUDSTENA',
  'COTTON WOOL & BUDSTENDERCARE',
  'COTTON WOOL & BUDSTIPPYS',
  'COTTON WOOL & BUDSVELVEX',
  'COTTON WOOL & BUDS',
  'DEODORANTS & ANTIPERSPIRANTSAXE',
  'DEODORANTS & ANTIPERSPIRANTSCHRIS ADAMS',
  'DEODORANTS & ANTIPERSPIRANTSFA',
  'DEODORANTS & ANTIPERSPIRANTSLURON',
  'DEODORANTS & ANTIPERSPIRANTSNIVEA',
  'DEODORANTS & ANTIPERSPIRANTSRASASI',
  'DEODORANTS & ANTIPERSPIRANTSREXONA',
  'DEODORANTS & ANTIPERSPIRANTS',
  'FACE CAREACNES',
  'FACE CAREARYUV',
  'FACE CAREBELLA',
  'FACE CAREDETTOL',
  'FACE CAREGARNIER',
  'FACE CAREHANAN',
  'FACE CARELA FEMME',
  'FACE CARELURON',
  'FACE CARENIVEA',
  'FACE CARESIMPLE',
  'FACE CAREST.IVES']